<a href="https://colab.research.google.com/github/LaurensSluyterman/PBPK-evaluation/blob/main/PBPK_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains all the necessary code in order to calcualte a confidence interval for a geometric mean ratio as is advocated in (insert name paper). 

In [26]:
#@title Make sure to run this first
import numpy as np
import scipy
from scipy import stats

def confidence_interval_individual_values(AUC_values, predicted_AUC_values, 
                                          alpha=0.1):
    if len(AUC_values) != len(predicted_AUC_values):
      print('different number of predictions and observations')
      return
    if alpha > 1:
      print('alpha cannot be larger than 1. For a 90% confidence interval, use alpha=0.1')
    z_values = np.log(predicted_AUC_values) - np.log(AUC_values)
    N = len(z_values)
    var = np.var(z_values)
    average = np.mean(z_values)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average - t * np.sqrt(var / N)
    logupperbound = average + t * np.sqrt(var / N)
    CI = [np.exp(loglowerbound), np.exp(logupperbound)]
    print(f'{100*(1-alpha)}% confidence interval: {CI}')

def confidence_interval(GM_observed, GM_predicted, CV, N, alpha=0.1):
    """CV must be the geometric coefficient of variation"""
    if CV>1:
      print('The coefficient of variation cannot exceed 1')
      return
    average = np.log(GM_observed)
    var = np.log(CV**2 + 1)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average + t * np.sqrt(var / N)
    logupperbound = average - t * np.sqrt(var / N)
    CI = [GM_predicted / np.exp(loglowerbound), GM_predicted / np.exp(logupperbound)]
    print(f'{100*(1-alpha)}% confidence interval: {CI}')


## Example when individual observations and predictions are available

The cell below illustrates how a confidence interval can be created using individual observations and predicitons. The hypothetical values can be replaced by relevant values. By default, a 90% confidence interval is given. To get, for instance, a 95% percent confidence interval, change alpha to 0.05.


In [27]:
hypothetical_observations = np.array([132,
                                      111,
                                      120,
                                      190,
                                      115,
                                      130])

hypothetical_predictions = np.array([110,
                                     121,
                                     125,
                                     170,
                                     125,
])

confidence_interval_individual_values(hypothetical_observations, 
                                      hypothetical_predictions,
                                      alpha=0.1)


different number of predictions and observations


# Example second approach

Quite often, only the geometric mean and coefficient of variation are published and not the individual AUC values. In this case the first approach is not possible and the second approach must be used.

In [15]:
hypothetical_observed_GM = 120
hypothetical_predicted_GM = 110
hypothetical_CV = 0.4 # N.B. this value should be between 0 and 1, not between 0 and 100!
Number_of_subjects = 20
confidence_interval(hypothetical_observed_GM, hypothetical_predicted_GM, 
                    hypothetical_CV, N=Number_of_subjects, alpha=0.1)

90.0% confidence interval: [0.789806013057465, 1.0639040015977201]
